# Тестовое задание

Для выполнения тестового задания требуется разработать модель, которая будет способна различать заголовки реальных и выдуманных новостей.
Для обучения модели используйте данные из файла `train.tsv`. В файле находится таблица, состоящая из двух колонок. 
В колонке title записан заголовок новости. В колонке is_fake содержатся метки: 0 – новость реальная; 1 – новость выдуманная.
Для демонстрации работы модели используйте данные тестового набора из файла `test.tsv`. В нем также есть колонка title, данные которой являются входными для вашей модели.
Вам нужно скопировать файл `test.tsv`, переименовать его в `predictions.tsv` и заполнить колонку is_fake значениями предсказаний вашей модели, аналогично `train.tsv`. 
Изначально колонка заполнена значением 0.

# Критерии оценки
1. Для оценки финального решения будет использоваться метрика F1 score.
2. Чистота кода, оформление и понятность исследования.

# Требования к решению
В качестве решения мы ожидаем zip-архив со всеми *.py и *.ipynb файлами в папке solution и файлом `predictions.tsv` в корне. Формат имени zip-архива: LastName_FirstName.zip (пример Ivanov_Ivan.zip).
Файл `predictions.tsv` должен включать в себя колонку title, содержащую те же данные, что и исходный файл `test.tsv`, а также колонку is_fake, содержащую значения 0 или 1.
Разметка тестового набора данных и включение его в обучение/валидацию запрещены.

В папке solution должно быть отражено исследование и весь код, необходимый для воспроизведения исследования.

Успехов!


# Решение

### Импортируем необходимые библиотеки

In [1]:
import numpy as np
import pandas as pd


import nltk
from nltk.stem.snowball import RussianStemmer
rus_stem = RussianStemmer()

import re

import matplotlib.pyplot as plt

import pymorphy2
morph = pymorphy2.MorphAnalyzer()



from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

In [2]:
#сменим директрию
import os

path_parent = os.path.dirname(os.getcwd())
os.chdir(path_parent)

### Загрузка данных

In [3]:
train = pd.read_csv("dataset/train.tsv", sep='\t')
test = pd.read_csv("dataset/test.tsv", sep='\t')

In [4]:
train.head()

,title,is_fake
0,Москвичу Владимиру Клутину пришёл счёт за вмеш...,1
1,Агент Кокорина назвал езду по встречке житейск...,0
2,Госдума рассмотрит возможность введения секрет...,1
3,ФАС заблокировала поставку скоростных трамваев...,0
4,Против Навального завели дело о недоносительст...,1


### Количество данных

In [5]:
print("Train Shape: %s"%str(train.shape))
print("Test Shape: %s"%str(test.shape))

Train Shape: (5758, 2)
Test Shape: (1000, 2)


### Предобработка текста

In [6]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/vasya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
stopwords = nltk.corpus.stopwords.words('russian')

In [8]:
print(stopwords)

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

Создадим колонку куда будем записывать все преобразования входных данных

In [9]:
test['total'] = test['title']
train['total'] = train['title']

In [10]:
train.head()

,title,is_fake,total
0,Москвичу Владимиру Клутину пришёл счёт за вмеш...,1,Москвичу Владимиру Клутину пришёл счёт за вмеш...
1,Агент Кокорина назвал езду по встречке житейск...,0,Агент Кокорина назвал езду по встречке житейск...
2,Госдума рассмотрит возможность введения секрет...,1,Госдума рассмотрит возможность введения секрет...
3,ФАС заблокировала поставку скоростных трамваев...,0,ФАС заблокировала поставку скоростных трамваев...
4,Против Навального завели дело о недоносительст...,1,Против Навального завели дело о недоносительст...


Функции для очистки, удаления стоп-слов и лемматизация

In [11]:
def clean_text(text):
    # оставляем только буквы, нижние подчеркивания и тире
    text = re.sub("[^а-яА-Яa-zA-Z\-\_ё]", " ", text)
    # удаляем лишние пробелы
    text = ' '.join(text.split())
    # приводим к нижнему регистру
    text = text.lower()
    return text

In [12]:
def remove_stopwords(text):
    no_stopword_text = [w for w in text.split() if not w in stopwords]
    return ' '.join(no_stopword_text)

In [13]:
def lemmatization_morph(text):
    '''Функция для леммантизации слов'''
    lemmatize_text = [morph.parse(w)[0].normal_form for w in text.split()]
    return ' '.join(lemmatize_text)

In [14]:
def stemmer(text):
    '''Функция для стемминга слов'''
    stemmer_text = [rus_stem.stem(w) for w in text.split()]
    return ' '.join(stemmer_text)

In [15]:
%%time
train['total'] = train['total'].apply(clean_text)
test['total'] = test['total'].apply(clean_text)

train['total'] = train['total'].apply(remove_stopwords)
test['total'] = test['total'].apply(remove_stopwords)

train['total'] = train['total'].apply(lemmatization_morph)
test['total'] = test['total'].apply(lemmatization_morph)

#train['total'] = train['total'].apply(stemmer)
#test['total'] = test['total'].apply(stemmer)

#Попробовав лемматизацию и стемминг понял что первое показывает результат лучше

CPU times: user 11.4 s, sys: 0 ns, total: 11.4 s
Wall time: 11.4 s


In [16]:
train.head()

,title,is_fake,total
0,Москвичу Владимиру Клутину пришёл счёт за вмеш...,1,москвич владимир клутина прийти счёт вмешатель...
1,Агент Кокорина назвал езду по встречке житейск...,0,агент кокорин назвать езда встречка житейский ...
2,Госдума рассмотрит возможность введения секрет...,1,госдума рассмотреть возможность введение секре...
3,ФАС заблокировала поставку скоростных трамваев...,0,фас заблокировать поставка скоростной трамвай ...
4,Против Навального завели дело о недоносительст...,1,против навальный завести дело недоносительство...


Разделим на обучающую выборку и тестовую

In [17]:
X_train, X_test, y_train, y_test = train_test_split(train['total'], train['is_fake'], test_size=0.20, random_state=0)

### Логистическая регрессия и TF-IDF векторизация

In [19]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2))    #включаем также биграммы
tfidf_vectorizer.fit(X_train)
tfidf_train = tfidf_vectorizer.transform(X_train)
tfidf_test  = tfidf_vectorizer.transform(X_test)

In [20]:
%%time
classifier = LogisticRegression(C=1e6)
classifier.fit(tfidf_train, y_train)

CPU times: user 2.82 s, sys: 4.84 s, total: 7.66 s
Wall time: 1.09 s


LogisticRegression(C=1000000.0)

In [21]:
pred_classifier_tfidf = classifier.predict(tfidf_test)
acc_classifier_tfidf = metrics.accuracy_score(y_test, pred_classifier_tfidf)
print(acc_classifier_tfidf)

0.859375


In [22]:
metrics.f1_score(y_test, pred_classifier_tfidf)

0.8663366336633662

### Логистическая регрессия и Count векторизация

In [23]:
count_vectorizer = CountVectorizer(ngram_range=(1, 2)) 
count_train = count_vectorizer.fit_transform(X_train)
count_test = count_vectorizer.transform(X_test)

In [24]:
classifier = LogisticRegression(C=1e6)
classifier.fit(count_train, y_train)
pred_classifier_count = classifier.predict(count_test)
acc_classifier_count = metrics.accuracy_score(y_test, pred_classifier_count)
print(acc_classifier_count)

0.8576388888888888


In [25]:
metrics.f1_score(y_test, pred_classifier_count)

0.8543516873889877

### Multinomial Naive Bayes  и Count векторизация

In [26]:
for alpha in np.arange(0.10, 1, .05):
    nb_classifier_tune = MultinomialNB(alpha=alpha)
    nb_classifier_tune.fit(count_train, y_train)
    pred_tune = nb_classifier_tune.predict(count_test)
    score = metrics.accuracy_score(y_test, pred_tune)
    print("Alpha: {:.2f} Score: {:.4f} f1-score: {:.4f}".format(alpha, score, metrics.f1_score(y_test, pred_tune)))

Alpha: 0.10 Score: 0.8602 f1-score: 0.8692
Alpha: 0.15 Score: 0.8628 f1-score: 0.8713
Alpha: 0.20 Score: 0.8628 f1-score: 0.8718
Alpha: 0.25 Score: 0.8663 f1-score: 0.8754
Alpha: 0.30 Score: 0.8689 f1-score: 0.8779
Alpha: 0.35 Score: 0.8689 f1-score: 0.8779
Alpha: 0.40 Score: 0.8689 f1-score: 0.8781
Alpha: 0.45 Score: 0.8655 f1-score: 0.8749
Alpha: 0.50 Score: 0.8628 f1-score: 0.8728
Alpha: 0.55 Score: 0.8637 f1-score: 0.8735
Alpha: 0.60 Score: 0.8628 f1-score: 0.8730
Alpha: 0.65 Score: 0.8628 f1-score: 0.8730
Alpha: 0.70 Score: 0.8628 f1-score: 0.8730
Alpha: 0.75 Score: 0.8646 f1-score: 0.8746
Alpha: 0.80 Score: 0.8655 f1-score: 0.8753
Alpha: 0.85 Score: 0.8663 f1-score: 0.8762
Alpha: 0.90 Score: 0.8672 f1-score: 0.8773
Alpha: 0.95 Score: 0.8672 f1-score: 0.8777


Лучше всего результат получается с MultinominalNB(alpha = 0.95), этот алгоритм и будем использовать для обучения модели на полном наборе данных

Попробуем ансамбли

In [27]:
sum_of_voices = pred_tune.astype(int) + pred_classifier_tfidf.astype(int)
voiting_result = (sum_of_voices > 1).astype(int)

In [28]:
accuracy_voiting = metrics.accuracy_score(voiting_result, y_test)
print(f'Voiting result: {accuracy_voiting}')

Voiting result: 0.8663194444444444


In [29]:
metrics.f1_score(y_test, voiting_result)

0.8705882352941176

## Обучение на полном наборе данных

Будем использовать алгоритм обучения MultinominalNB(alpha = 0.95) тк он показал себя наилучшим образом

In [30]:
X_train_full, y_train_full = train['total'], train['is_fake'] 

In [31]:
count_vectorizer = CountVectorizer(ngram_range=(1, 2)) 
count_train_full = count_vectorizer.fit_transform(X_train_full)
count_test_full  = count_vectorizer.transform(test['total'])

In [32]:
nb_classifier_tune = MultinomialNB(alpha=0.95)
nb_classifier_tune.fit(count_train_full, y_train_full)
final_prediction = nb_classifier_tune.predict(count_test_full)

In [33]:
output = pd.DataFrame({'title': test['title'],
                      'is_fake': final_prediction})

output.to_csv('predictions.tsv', index=False, sep='\t')
print(output)

                                                 title  is_fake
0    Роскомнадзор представил реестр сочетаний цвето...        1
1    Ночью под Минском на президентской горе Белара...        1
2    Бывший спичрайтер Юрия Лозы рассказал о трудно...        1
3    Сельская церковь, собравшая рекордно низкое ко...        1
4    Акции Google рухнули после объявления о переза...        0
..                                                 ...      ...
995  Прокуратура заподозрила Явлинского в авторитар...        1
996  В День Победы стратегические ракетоносцы Ту-16...        1
997  СК возбудил дело против авиакомпании «Победа» ...        1
998  Криптомонетный двор Туркменистана выпустил юби...        1
999  Deutsche Bahn заплатит рекордный штраф за чтен...        1

[1000 rows x 2 columns]
